<a href="https://colab.research.google.com/github/kimsejin111/speedlimit_classification/blob/main/speedlimits.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import PIL
from PIL import Image
import time
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='3'
import pathlib

In [ ]:
data_dir='/content/gdrive/MyDrive/data'

In [ ]:
data_dir=pathlib.Path(data_dir)

batch_size=32
image_height=180
image_width=180

d_image_count = len(list(data_dir.glob('*/*.png')))
print(d_image_count)


In [ ]:
train_ds=tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2, # 왜 0.2로 하는거지?
    subset="training",
    seed=123,   # seed는 무얼 의미하는 거지?
    image_size=(image_height, image_width),
    batch_size=batch_size
)
val_ds=tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(image_height, image_width),
    batch_size=batch_size
)

In [ ]:
class_names = train_ds.class_names
plt.figure(figsize=(10,10))
for images, labels in train_ds.take(1): #take 함수는 무슨 함수일까
    for i in range(9):
        ax=plt.subplot(3,3,i+1)
        plt.imshow(images[i].numpy().astype("uint8")) #numpy,imshow 는 무슨 함수일까
        plt.title(class_names[labels[i]])
        plt.axis("off")
plt.show()

In [ ]:
AUTOTUNE =tf.data.experimental.AUTOTUNE
train_ds=train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds=val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
normaliztion_layer= layers.experimental.preprocessing.Rescaling(1./255)

normalized_ds=train_ds.map(lambda x,y:(normaliztion_layer(x),y))
image_batch,labels_batch = next(iter(normalized_ds)) #자동 반복 함수
first_image = image_batch[0]
print(np.min(first_image), np.max(first_image))

In [ ]:
data_augmentation = keras.Sequential(
    [
        layers.experimental.preprocessing.RandomFlip(
            "horizontal",
            input_shape=(
                image_height,
                image_width,
                3)),
        layers.experimental.preprocessing.RandomRotation(0.1),
        layers.experimental.preprocessing.RandomZoom(0.1),
    ]
)

In [ ]:
num_classes=5

model2 =Sequential([
    layers.experimental.preprocessing.Rescaling(1. / 255, input_shape=(image_height, image_width, 3)),
    layers.Conv2D(16, 3, padding='same', activation='relu'),
    layers.MaxPool2D(),
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPool2D(),
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPool2D(),  # Pooling 필터링 개념으로 생각하자  해당 convolution에서 MAX값으로 필터링 하여 데이터 크기를 줄임
    layers.Dropout(0.2),
    layers.Flatten(),  # convolution의 dimension을 줄여주는 함수
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes)
])

In [ ]:
model2.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
epochs=15
history=model2.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs,
)

In [ ]:
acc_E=history.history['accuracy']
val_E_acc=history.history['val_accuracy']

loss_E=history.history['loss']
val_E_loss=history.history['val_loss']

epochs_range= range(epochs)

plt.figure(figsize=(8,8))
plt.subplot(1,2,1)
plt.plot(epochs_range,acc_E,label="Training ACC")
plt.plot(epochs_range,val_E_acc,label="Validation ACC")
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1,2,2)
plt.plot(epochs_range,loss_E,label="Training Loss")
plt.plot(epochs_range,val_E_loss,label="Validation Loss")
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')

plt.show()